### Oracle Vector DB wrapped as a llama-index custom Vector Store

* ispired by: https://docs.llamaindex.ai/en/stable/examples/low_level/vector_store.html

In [1]:
import logging
import sys

from typing import List, Any, Optional, Dict, Tuple
from llama_index.vector_stores.types import (
    VectorStore,
    VectorStoreQuery,
    VectorStoreQueryResult,
)
from llama_index import StorageContext, VectorStoreIndex, ServiceContext
from llama_index.schema import TextNode, BaseNode, Document

import oci
import ads

# only
import oracledb
from oci_utils import load_oci_config
from ads.llm import GenerativeAIEmbeddings, GenerativeAI
from oracle_vector_db import OracleVectorStore

from config import EMBED_MODEL
from config_private import COMPARTMENT_OCID, ENDPOINT

In [2]:
# version I'm using
print(f"oracledb version: {oracledb.__version__}")
print(f"oci version: {oci.__version__}")

oracledb version: 2.0.0.dev20231121
oci version: 2.112.1+preview.1.1649


In [3]:
# for debugging
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
# setup
oci_config = load_oci_config()

# need to do this way
api_keys_config = ads.auth.api_keys(oci_config)

# english, or for other language use: multilingual

embed_model = GenerativeAIEmbeddings(
    compartment_id=COMPARTMENT_OCID,
    model=EMBED_MODEL,
    auth=ads.auth.api_keys(oci_config),
    # Optionally you can specify keyword arguments for the OCI client, e.g. service_endpoint.
    client_kwargs={"service_endpoint": ENDPOINT},
)

#### Using the wrapper for the DB Vector Store

In [5]:
v_store = OracleVectorStore(verbose=True)

In [6]:
question = (
    "What is JSON Relational Duality in Oracle Database 23c? Explain with details"
)

In [7]:
# embed the query using OCI GenAI
query_embedding = embed_model.embed_documents([question])[0]

#  wrap in llama-index
query_obj = VectorStoreQuery(query_embedding=query_embedding, similarity_top_k=6)

#### Use our Vector Store DB

In [8]:
%%time

q_result = v_store.query(query_obj)

2024-01-04 18:28:26,026 - INFO - ---> Calling query on DB
2024-01-04 18:28:26,371 - INFO - select: select V.id, C.CHUNK, C.PAGE_NUM, 
                            ROUND(VECTOR_DISTANCE(V.VEC, :1, DOT), 3) as d,
                            NAME 
                            from VECTORS V, CHUNKS C, BOOKS B
                            where C.ID = V.ID and
                            C.BOOK_ID = B.ID
                            order by d
                            FETCH FIRST 6 ROWS ONLY
2024-01-04 18:28:26,696 - INFO - Query duration: 0.7 sec.


CPU times: user 19.8 ms, sys: 5.83 ms, total: 25.6 ms
Wall time: 674 ms


In [9]:
for n, id, sim in zip(q_result.nodes, q_result.ids, q_result.similarities):
    print(f"Dod. id: {id}")
    print(f"Similarity: {-sim}")
    print(n.text)
    print("")

Dod. id: bfbaa15634643a2aac6e77b85a4f8a335eba697d741aaaa3646bdfb07fa3ba22
Similarity: 0.696
2 Application Development JSON JSON-Relational Duality JSON Relational Duality Views are fully updatable JSON views over relational data. Data is still stored in relational tables in a highly efficient normalized format but can be accessed by applications in the form of JSON documents. Duality views provide you with game-changing flexibility and simplicity by overcoming the historical challenges developers have faced when building applications using relational or document models. Related Resources View Documentation JSON Schema JSON Schema-based validation is allowed with the SQL condition IS JSON and with a PL/SQL utility function. A JSON schema is a JSON document that specifies allowed properties (field names) and the corresponding allowed data types, and whether they are optional or mandatory. By default, JSON data is schemaless, providing flexibility. However, you may want to ensure that you

#### Integrate in the bigger RAG picture

In [10]:
llm_oci = GenerativeAI(
    compartment_id=COMPARTMENT_OCID,
    max_tokens=1024,
    # Optionally you can specify keyword arguments for the OCI client, e.g. service_endpoint.
    client_kwargs={"service_endpoint": ENDPOINT},
)

In [11]:
service_context = ServiceContext.from_defaults(llm=llm_oci, embed_model=embed_model)

In [12]:
index = VectorStoreIndex.from_vector_store(
    vector_store=v_store, service_context=service_context
)

In [13]:
query_engine = index.as_query_engine(similarity_top_k=5)

In [14]:
%%time

response = query_engine.query(question)

print(f"Question: {question}")
print(response.response)
print("")

2024-01-04 18:28:27,919 - INFO - ---> Calling query on DB
2024-01-04 18:28:28,098 - INFO - select: select V.id, C.CHUNK, C.PAGE_NUM, 
                            ROUND(VECTOR_DISTANCE(V.VEC, :1, DOT), 3) as d,
                            NAME 
                            from VECTORS V, CHUNKS C, BOOKS B
                            where C.ID = V.ID and
                            C.BOOK_ID = B.ID
                            order by d
                            FETCH FIRST 5 ROWS ONLY
2024-01-04 18:28:29,718 - INFO - Query duration: 1.8 sec.


Question: What is JSON Relational Duality in Oracle Database 23c? Explain with details
JSON Relational Duality is a feature in Oracle Database 23c that allows developers to access data stored in relational tables in the form of JSON documents. This is done by creating Duality views, which are fully updatable JSON views over relational data. 

With JSON Relational Duality, developers can take advantage of the best features of both the relational and document models. The relational model is simple, structured, and easy to understand, while the document model offers flexibility and simplicity for developing applications. 

Duality views overcome the challenges of building applications using relational or document models by allowing developers to work with data in a format that best suits their needs. This can increase productivity and flexibility in application development. 

Would you like to know more about the benefits of using JSON Relational Duality in Oracle Database 23c? 

CPU time

#### Additional tests